In [38]:
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader , Dataset
import matplotlib.pyplot as plt
import torch.optim as optim


# Loading images to PyTorch

In [4]:
train_transforms = transforms.Compose([
transforms.ToTensor(),
transforms.Resize((128, 128)),
])

# Load  dataset
dataset_train = ImageFolder(
"D:\hossam\Deep learnig\Intermediate Deep Learning with PyTorch\clouds\clouds_train",
transform=train_transforms,
)

In [6]:
# Create DataLoaders to iterate through batches
dataloader_train = DataLoader(
                        dataset_train,
                        shuffle=True,
                        batch_size=1,
                        )

image, label = next(iter(dataloader_train))
print(image.shape)

torch.Size([1, 3, 128, 128])


In [46]:
image = image.squeeze().permute(1, 2, 0)
print(image.shape)

torch.Size([128, 3, 128])


# Data augmentation

In [3]:
#Data augmentation: Generating more data by applying random transformations to original images
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45),
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    ])


dataset_train = ImageFolder(
"D:\hossam\Deep learnig\Intermediate Deep Learning with PyTorch\clouds\clouds_train",
transform=train_transforms,
)

# CNN Convolutional Neural Networks

In [58]:
import torch.nn as nn 
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Define feature extractor
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
        )
        # Define classifier
        self.classifier = nn.Linear(64*16*16, num_classes)
    
    def forward(self, x):  
        # Pass input through feature extractor and classifier
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

In [9]:
# Data augmentation 
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAutocontrast(),
    transforms.RandomRotation(45),
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    ])

In [131]:
# read our train data 
dataset_train = ImageFolder(
"D:\hossam\Deep learnig\Intermediate Deep Learning with PyTorch\clouds\clouds_train",
transform=train_transforms,
)

# read our test data 
dataset_test = ImageFolder(
"D:\hossam\Deep learnig\Intermediate Deep Learning with PyTorch\clouds\clouds_test",
transform=train_transforms,
)

In [70]:
# Create DataLoaders to iterate through batches
dataloader_train = DataLoader(
                        dataset_train,
                        shuffle=True,
                        batch_size=1,
                        )

dataloader_test = DataLoader(
                        dataset_train,
                        shuffle=True,
                        batch_size=1,
                        )

In [17]:
image, label = next(iter(dataloader_train))
print(image.shape)

torch.Size([1, 3, 128, 128])


In [19]:
image = image.squeeze().permute(1, 2, 0)
print(image.shape)

torch.Size([128, 128, 3])


# Image classifier training loop

In [62]:
net = Net(num_classes=7)

# Image classifier training loop
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
for epoch in range(10):
    running_loss = 0.0
    for images, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [66]:
epoch_loss = running_loss / len(dataloader_train)
print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

Epoch 10, Loss: 0.0000


<h1>Multi-class model evaluation</h1>



<p style="font-size:18px;">
Let's evaluate our cloud classifier with precision and recall to see how well it can classify the seven cloud types. In this multi-class classification task it is important how you average the scores over classes. Recall that there are four approaches:
</p>

<h3>1. Not averaging</h3>
<p>Analyze the results per class individually.</p>

<h3>2. Micro-averaging</h3>
<p>Ignore the classes and compute the metrics globally.</p>

<h3>3. Macro-averaging</h3>
<p>Compute metrics per class and then average them.</p>

<h3>4. Weighted-averaging</h3>
<p>Like macro-averaging, but the average is weighted by class size.</p>


In [92]:
from torchmetrics import Recall , Precision
import torch

In [84]:
recall_per_class = Recall(task="multiclass", num_classes=7, average=None)
recall_micro = Recall(task="multiclass", num_classes=7, average="micro")
recall_macro = Recall(task="multiclass", num_classes=7, average="macro")
recall_weighted = Recall(task="multiclass", num_classes=7, average="weighted")

In [111]:
# Define metrics
metric_precision = Precision(task='multiclass', num_classes=7, average=None)
metric_recall = Recall(task='multiclass', num_classes=7, average=None)

In [113]:
# Define metrics
metric_precision_micro = Precision(task='multiclass', num_classes=7, average='micro')
metric_recall_micro = Recall(task='multiclass', num_classes=7, average='micro')

In [115]:
# Define metrics
metric_precision_macro = Precision(task='multiclass', num_classes=7, average='macro')
metric_recall_macro = Recall(task='multiclass', num_classes=7, average='macro')

In [117]:
# Define metrics
metric_precision_weighted = Precision(task='multiclass', num_classes=7, average='weighted')
metric_recall_weighted = Recall(task='multiclass', num_classes=7, average='weighted')

In [165]:
# evaluate Not AVG
net.eval()
with torch.no_grad():
    for images, labels in dataloader_test:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        metric_precision(preds, labels)
        metric_recall(preds, labels)

precision = metric_precision.compute()
recall = metric_recall.compute()
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Precision: tensor([0.5000, 0.6222, 0.7692, 0.6071, 0.6667, 0.2711, 0.4000])
Recall: tensor([0.1818, 0.9333, 0.7143, 0.3736, 0.6015, 0.9184, 0.0328])


In [167]:
# evaluate micro
net.eval()
with torch.no_grad():
    for images, labels in dataloader_test:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        metric_precision_micro(preds, labels)
        metric_recall_micro(preds, labels)

precision_micro = metric_precision_micro.compute()
recall_micro = metric_recall_micro.compute()
print(f"Precision: {precision_micro}")
print(f"Recall: {recall_micro}")

Precision: 0.5126582384109497
Recall: 0.5126582384109497


In [169]:
# evaluate Macro
net.eval()
with torch.no_grad():
    for images, labels in dataloader_test:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        metric_precision_macro(preds, labels)
        metric_recall_macro(preds, labels)

precision_macro = metric_precision_macro.compute()
recall_macro = metric_recall_macro.compute()
print(f"Precision: {precision_macro}")
print(f"Recall: {recall_macro}")

Precision: 0.5966891646385193
Recall: 0.5579268932342529


In [170]:
# evaluate metric weighted 
net.eval()
with torch.no_grad():
    for images, labels in dataloader_test:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        metric_precision_weighted(preds, labels)
        metric_recall_weighted(preds, labels)

precision_micro = metric_precision_micro.compute()
recall_micro = metric_recall_micro.compute()
print(f"Precision: {precision_micro}")
print(f"Recall: {recall_micro}")

Precision: 0.5126582384109497
Recall: 0.5126582384109497


# Analyzing performance per class 

In [179]:
# Get precision per class
precision_per_class = {
    k: precision[v].item()  # Ensure 'precision' is lowercase
    for k, v 
    in dataset_test.class_to_idx.items()
}
print(precision_per_class)

{'cirriform clouds': 0.5, 'clear sky': 0.6222222447395325, 'cumulonimbus clouds': 0.7692307829856873, 'cumulus clouds': 0.6071428656578064, 'high cumuliform clouds': 0.6666666865348816, 'stratiform clouds': 0.27108433842658997, 'stratocumulus clouds': 0.4000000059604645}
